# Python Tricks 
              Dan Bader 

In [ ]:
# price amount in cents to avoid currency rounding
shoes = {'name': 'Fancy Shoes', 'price': 14900} 

def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']
    return price

apply_discount(shoes, 0.25), apply_discount(shoes, 0.5), apply_discount(shoes, 1.0), apply_discount(shoes, 1.5)

Assertions are meant to be *internal self-checks*. Pythons's assert statement is a debugging aid, not a mechanism for handling run-time errors.

In [ ]:
# Assertion implementation
if __debug__:
    if not expression1:
        raise AssertionError(expression2)


__debug__ is a global variable which is **true** under normal circumstances and **false** if optimizations are requested.

In computer programming jargon, a **heisenbug** is a software bug that seems to disappear or alter its behavior when one attempts to study it.

Assertions can be globally disabled with -O and -OO command lin switches as wll as `PYTHONOPTIMIZE` environment variable in CPython.

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(),
    assert store.has_product(prod_id),
    store.get_product(prod_id).delete()

In [ ]:
# Better version
def delete_product(prod_id, user):
    if not user.is_admin():
        raise AuthError('Must be admin to delete')
    if not store.has_product(prod_id):
        raise ValueError('Unknow product id')
    store.get_product(prod_id).delete()

In [9]:
assert(1 == 2, 'This should fail')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-9-2c057bd7fe24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'This should fail')


In [10]:
assert 1 == 2, "this should fail"

AssertionError: this should fail

In [11]:
names = [
    'Alice',
    'Bob',
    'Dilbert'
    'Jane'
]
print(names)

['Alice', 'Bob', 'DilbertJane']


In Python, a comma can be placed after every item in a list, dict, or set constant, including the last item.

## Context Manager and with statement

`with` statement simplifies some common resource management patterns by abstracting their functionality and allowing them to be factored out and reused.

Another good example where the `with` statement is used effectively is `threading.Lock` class.

In [ ]:
with open('hello.txt', 'w') as f:
    f.write('Hello World!!')

A class can support `with` statement by implementing so-called _context managers_. Context manager is a simple protocol/interface that your object needs to follow in order to support `with` statement. 

A class/object must add `__enter__` and `__exit__` methods it it want to function as a context manager.


In [ ]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [ ]:
with ManagedFile('hello.txt') as f:
    f.write('hello world')
    f.write('good byte')

The `contextlib` utility module in the standard library provides a few more abstractions built on top of the basic context manager protocol.

In [12]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):  # generator()
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()
        
with managed_file('hello.txt') as f:
    f.write('hello and byte')

In [14]:
class Indenter:
    def __init__(self):
        self.level = 0
        self.spacing = [
            '....',
            '****',
            '++++',
        ]
    
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, text):
        print('....' * self.level + text)
        

with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello!!')
        with indent:
            indent.print('bonjour')
    indent.print('bye')

....hi!
........hello!!
............bonjour
....bye


Excercise: implementing a context manager that measures the execution time of a code block using the time.time function. Be sure to try out writ ing both a decorator-based and a class-based variant to drive home the difference between the two